# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,patiala,76.4003,30.3267,293.12,51,0,1.54,IN,1728780030
1,1,adamstown,-130.1015,-25.0660,294.45,79,98,4.26,PN,1728780032
2,2,edinburgh of the seven seas,-12.3116,-37.0676,286.19,72,10,5.17,SH,1728780033
3,3,port elizabeth,25.5701,-33.9180,291.31,96,75,6.69,ZA,1728780035
4,4,olonkinbyen,-8.7187,70.9221,271.62,96,100,13.33,SJ,1728780037


In [11]:
city_data_df.columns

Index(['City_ID', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date'],
      dtype='object')

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lat",
    "Lng",
    geo = True,
    tiles = "OSM",
    frame_width = 500,
    frame_height = 300,
    size = "Humidity",
    scale = 0.75,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lat,Lng]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [33]:
# Narrow down cities that fit criteria and drop any results with null values

## Define criteria of my dream vacation place
criteria = {}
criteria["min_temp"] = 297.039 ## in Kelvin (75 F)
criteria["max_temp"] = 310.928 ## in Kelvin (100 F)
criteria["humidity"] = 50

ideal_vacation_df = city_data_df.loc[(city_data_df["Max Temp"] >= criteria["min_temp"]) & (city_data_df["Max Temp"] <= criteria["max_temp"])& (city_data_df["Humidity"] <= criteria["humidity"])]

# Drop any rows with null values
ideal_vacation_df = ideal_vacation_df.dropna(axis=0)

# Display sample data
ideal_vacation_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
50,50,alice springs,133.8833,-23.7000,301.90,28,0,6.17,AU,1728780120
80,80,clodomira,-64.1311,-27.5744,298.33,46,72,4.85,AR,1728780173
120,120,ghadamis,9.5007,30.1337,300.49,43,0,4.51,LY,1728780246
124,124,oriximina,-55.8661,-1.7656,304.05,44,94,1.36,BR,1728780256
132,132,floresville,-98.1561,29.1336,302.52,40,0,3.75,US,1728780270


### Step 3: Create a new DataFrame called `hotel_df`.

In [35]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_vacation_df.copy()
hotel_df = hotel_df.drop(['City_ID', 'Cloudiness', 'Wind Speed', 'Date'], axis=1)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = {}

# Display sample data
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Country,Hotel Name
50,alice springs,133.8833,-23.7000,301.90,28,AU,NaN
80,clodomira,-64.1311,-27.5744,298.33,46,AR,NaN
120,ghadamis,9.5007,30.1337,300.49,43,LY,NaN
124,oriximina,-55.8661,-1.7656,304.05,44,BR,NaN
132,floresville,-98.1561,29.1336,302.52,40,US,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [40]:
# Set parameters to search for a hotel
radius = 100000 ## I set the radius to 100,000 instead of 10,000 because a radius of 10,000 was not yielding any hotels
# Parameters for APi request (add filter and bias conditions within loop below)
# "filter":f"circle:{longitude},{latitude},{radius}", 
# "bias": f"proximity:{longitude},{latitude}", 
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key, 
    "limit": 5
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
alice springs - nearest hotel: No hotel found
clodomira - nearest hotel: No hotel found
ghadamis - nearest hotel: Peace Hotel
oriximina - nearest hotel: No hotel found
floresville - nearest hotel: No hotel found
humaita - nearest hotel: No hotel found
atar - nearest hotel: No hotel found
bom jesus - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
nguigmi - nearest hotel: No hotel found
sur - nearest hotel: No hotel found
alaghsas - nearest hotel: Auberge Princessa
la junta - nearest hotel: No hotel found
port hedland - nearest hotel: No hotel found
as sulayyil - nearest hotel: Хотел Војводина
serra talhada - nearest hotel: No hotel found
awjilah - nearest hotel: No hotel found
chino valley - nearest hotel: No hotel found
imperial - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found
dongola - nearest hotel: No hotel found
karratha - nearest hotel: No hotel found
al kharj - nearest hotel: Cabana Prestige
illizi - nearest hote

,City,Lat,Lng,Max Temp,Humidity,Country,Hotel Name
50,alice springs,133.8833,-23.7000,301.90,28,AU,No hotel found
80,clodomira,-64.1311,-27.5744,298.33,46,AR,No hotel found
120,ghadamis,9.5007,30.1337,300.49,43,LY,Peace Hotel
124,oriximina,-55.8661,-1.7656,304.05,44,BR,No hotel found
132,floresville,-98.1561,29.1336,302.52,40,US,No hotel found
142,humaita,-63.0208,-7.5061,302.62,37,BR,No hotel found
175,atar,-13.0499,20.5169,308.77,20,MR,No hotel found
180,bom jesus,-44.3586,-9.0744,301.87,27,BR,No hotel found
183,crane,-102.3501,31.3974,300.59,16,US,No hotel found
232,nguigmi,13.1092,14.2495,303.71,22,NE,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [43]:
%%capture --no-display

# Configure the map plot
# Information on how to add additional fields to hover message by adjusting hover_cols parameter from Google AI 
map_plot_2 = hotel_df.hvplot.points(
    "Lat",
    "Lng",
    geo = True,
    tiles = "OSM",
    frame_width = 500,
    frame_height = 300,
    scale = 0.75,
    color = "City",
    hover_cols=['Lat', 'Lng', 'City', 'Country', 'Hotel Name']
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lat,Lng]   (City,Country,Hotel Name)